In [9]:
import allel
from numba import njit
import malariagen_data
import pandas as pd
import numpy as np
%matplotlib inline

import h5py
import zarr

import locusPocus

### Writing out haplotypes for phylogenetic analysis

First, lets set the genome position of our focus, and also set a position for a separate analysis of a downstream and upstream region. As the coeae1f signal is so large, I set this to be 2 megabases away. We can also set the size of the flanking region in bp, I set this to 10kb either side of the focus (same as Xavi rdl ms).  

In [10]:
region_names = ['focal', 'upstream', 'downstream', 'collinear']
regions = [28_545_767, 26_545_767, 30_545_767, 45_000_000]

locus_name = 'coeae1f'
contig = '2L'
flanking = 10_000

out_prefix = "results/phylo" 

outgroup_h5_path = "/home/sanj/ag1000g/data/outgroups/outgroup_alleles.h5"
phase1_positions_array_path = f"/home/sanj/ag1000g/data/ag1000g.phase1.ar3.pass/{contig}/variants/POS/"
remove_2la_hets = True ### only if analysing inside 2la region

sample_sets = [
    # Ag1000G phase 3 sample sets in Ag3.0
    "AG1000G-GH", 
    'AG1000G-ML-A',
     'AG1000G-BF-A',
     'AG1000G-BF-B',
     'AG1000G-GN-A',
     'AG1000G-GN-B',
    'AG1000G-TZ',
    # Amenta-Etego sample sets in Ag3.3
    # GAARDIAN sample set in Ag3.4
    '1244-VO-GH-YAWSON-VMF00149',
    # GAARD Ghana sample set in Ag3.2
     "1244-VO-GH-YAWSON-VMF00051",
     '1245-VO-CI-CONSTANT-VMF00054',
     '1253-VO-TG-DJOGBENOU-VMF00052',
     '1237-VO-BJ-DJOGBENOU-VMF00050'
]


Then lets select which cohorts we want to load, connect to Ag3() API and load some metadata.

In [11]:
# Define region strings
region_dict = dict(zip(region_names, regions))

for k, v in region_dict.items():
    region_dict[k] = f'{contig}:{v-flanking}-{v+flanking}'

ag3 = malariagen_data.Ag3(pre=True)

In [12]:
metadata = ag3.sample_metadata(sample_sets)
metadata['aim_species'].value_counts()

Load sample metadata:   0%|          | 0/12 [00:00<?, ?it/s]

gambiae                          1146
coluzzii                         1041
arabiensis                        228
intermediate_gambiae_coluzzii      16
Name: aim_species, dtype: int64

### Write haplotypes to FASTA

In this notebook, we will construct FASTA sequences (or a multi-sequence alignment) from haplotypes in selected cohorts of the ag3, plus some outgroups. We will use *An. merus*, *An. melas*, and *An. quadriannulatus* as outgroups. 

In [13]:
outgroup_alleles = h5py.File(outgroup_h5_path, 'r')
outgroup_names = ['meru', 'mela', 'quad'] # we will use these only 

phase1_pos = zarr.open_array(phase1_positions_array_path, mode='r')
phase1_pos = allel.SortedIndex(phase1_pos)

If we are analysing a region within 2La, we will also restrict analyses to individuals homozygous for either 2la or 2l+a, to aid interpretation. 

In [14]:
if remove_2la_hets:
    path_to_compkaryo_table = "/home/sanj/projects/gaardian/results/gaard_and_ag3.2la.karyo.tsv" # must be for all samples and cohorts in this analysis
    
    karyos = pd.read_csv(path_to_compkaryo_table, sep="\t", index_col=0)
    hom_2la_partner_sample_ids = karyos.query("mean_genotype < 0.1 | mean_genotype > 1.8")['partner_sample_id']
    print(f"There are {len(hom_2la_partner_sample_ids)} 2la/2l+a homozygote individuals")

    karyo_df = karyos.query("mean_genotype < 0.1 | mean_genotype > 1.8")[['partner_sample_id', 'mean_genotype']]
    karyo_df = karyo_df.rename(columns={'partner_sample_id':'sample_id'})

There are 1683 2la/2l+a homozygote individuals


In [15]:
@njit
def haps_to_fasta_numba(haplos, var_alleles):
    """
    Loop through haplotypes and ref/alt alleles to construct a haplotype FASTA. Numba accelerated.
    """
    seq_list = [] 
    for i in range(haplos.shape[1]):
        seq_arr = []

        for gn, allele in enumerate(haplos[:,i]):
            seq_arr.append(var_alleles[gn][allele])
        
        seq_list.append(np.array(seq_arr))
    return(seq_list)

def haps_to_fasta_df(haps):
    """
    Take haplotype XArray and create pandas df of fasta sequences
    """
    # transform xarray into haplotype array
    haplos = allel.GenotypeArray(haps['call_genotype'].compute()).to_haplotypes()
    # extract ref and alt alleles array
    var_alleles = haps['variant_allele'].compute()
    
    seq_arr = haps_to_fasta_numba(haplos.values, var_alleles.values)
    seq_arr = np.vstack(seq_arr)
    
    # Make dataframe of all haplotype sequences for region
    fasta_df = pd.DataFrame(seq_arr)
    fasta_df.columns = haps['variant_position'].compute().values
    sample_ids = haps['sample_id'].compute().values
    fasta_df.loc[:, 'hap'] = ["> " + p for p in locusPocus.rename_duplicates(np.repeat(sample_ids, 2))]
    cols = list(fasta_df)
    # move the column to head of list using index, pop and insert
    cols.insert(0, cols.pop(cols.index('hap')))
    fasta_df = fasta_df.loc[:, cols]
    return(fasta_df)

def remove_missing_invariant_fasta_df(fasta_df, remove_invariant=False):
    """
    Remove any columns in pandas dataframe that are missing genotypes ('.')
    """
    missing_bool = fasta_df.apply(lambda x: any(x == b'.') , axis=0)
    print(f"Removing {missing_bool.sum()} alleles with a missing call")
    fasta_df = fasta_df.loc[:, ~missing_bool]
    fasta_df = fasta_df.set_index('hap')
    
    if remove_invariant:
        invariant_cols = fasta_df.nunique() <= 1
        print(f"Removing {invariant_cols.sum()} invariant SNPs")
        fasta_df = fasta_df.loc[:, ~invariant_cols]
        print(f"There are {fasta_df.shape[0]} haplotypes and {fasta_df.shape[1]} segregating haplotype calls")

    fasta_df.loc[:, 'seq'] = fasta_df.apply(lambda x: b''.join(x).decode('utf8'), axis=1)
    return(fasta_df.reset_index()[['hap', 'seq']])

Optionally, add sweep ID columns to metadata to allow for visualise in R 

In [16]:
sweep_ids = True

if sweep_ids:
    meta_sweeps = pd.read_csv("../../results/haplotype_metahaps.tsv", sep="\t")
    meta_sweeps = meta_sweeps[['sample_id', 'hap_cluster']]

For each region, convert the haps to fasta and write to file! 

In [17]:
out_prefix = "../../results/phylo" 

In [23]:
haps = {}
for name, region in region_dict.items():
    # Load haplotypes for region, find intersection with phase 1 outgroup data
    print(f"loading haplotypes | {name} | {region}")
    haps[name] = ag3.haplotypes(region=region, sample_sets=sample_sets, analysis='gamb_colu_arab')
    print(name, region, haps[name]['call_genotype'].shape)
    sample_ids = haps[name]['sample_id'].compute().values
    outgroup_region_bool, phase1_bool = phase1_pos.locate_intersection(haps[name]['variant_position'].compute().values)
    haps[name] = haps[name].sel(variants=phase1_bool)
    
    if remove_2la_hets:
    ### for 2La ###
        hom_2la_bool = np.isin(sample_ids, hom_2la_partner_sample_ids)
        haps[name] = haps[name].sel(samples=hom_2la_bool)
    
    # for each haplotype, loop through SNPs and create a FASTA sequence array depending on alleles
    print(f"Converting haps to FASTA sequence | {name}")
    fasta_df = haps_to_fasta_df(haps[name])

    # concat outgroup sequences
    outgroup_contig = {}
    for out in outgroup_names:
        outgroup_contig[out] = outgroup_alleles[contig][out][:][outgroup_region_bool]
        df = pd.DataFrame(outgroup_contig[out]).T
        df.columns = haps[name]['variant_position'].compute().values
        df.loc[:, 'hap'] = "> " + out
        fasta_df = pd.concat([fasta_df, df], axis=0)
    
    # remove missing alleles in the outgroups and invariant sites 
    fasta_df = remove_missing_invariant_fasta_df(fasta_df, remove_invariant=False)
    
    # write to csv with \n sep to make FASTA file
    fasta_df.to_csv(f"{out_prefix}/{locus_name}_{name}.fasta", sep="\n", index=False, header=False)
    print(f'Multiple alignment FASTA written \n')
    
    # Make an artificial metadata table for the outgroups and concat to the bottom of normal metahaps
    outgroup_n = len(outgroup_names)
    outgroup_meta = metadata.iloc[:outgroup_n, :].copy()
    outgroup_meta.iloc[:outgroup_n, :] = "Nan"
    for i, out in enumerate(outgroup_names):
        outgroup_meta.loc[i, 'sample_id'] = out
        outgroup_meta.loc[i, 'aim_species'] = out
        outgroup_meta.loc[i, 'cluster_id'] = out
        
    metahaps, q = locusPocus.load_metahaps(sample_sets, sample_ids)
    if sweep_ids: metahaps.loc[:, 'hap_cluster'] = meta_sweeps.loc[:, 'hap_cluster']
    if remove_2la_hets:
        metahaps = metahaps.query("sample_id in @hom_2la_partner_sample_ids")
    meta = pd.concat([metahaps, outgroup_meta], axis=0)
    
    if remove_2la_hets:
        meta = meta.merge(karyo_df, how='left')
        meta.loc[:, 'karyotype'] = np.where(
             meta['mean_genotype'].between(0, 0.4, inclusive="both"), 
            '2l+a', 
             np.where(
                meta['mean_genotype'].between(1.6, 2, inclusive="both"), '2la', 'Unknown'
             )
        )
    
    # remove > and join with metadata for each pop, useful for plotting phylo trees metadata
    fasta_df.loc[:, 'hap'] = fasta_df['hap'].str.strip('> ')
    all_haps = pd.concat([fasta_df.reset_index(drop=True), meta.reset_index(drop=True)], axis=1)
    all_haps.to_csv(f"{out_prefix}/{locus_name}_{name}.metadata.tsv", sep="\t", index=False, header=True)

loading haplotypes | focal | 2L:28535767-28555767
focal 2L:28535767-28555767 (6819, 2431, 2)
Converting haps to FASTA sequence | focal
Removing 201 alleles with a missing call
Multiple alignment FASTA written 

loading haplotypes | upstream | 2L:26535767-26555767
upstream 2L:26535767-26555767 (4359, 2431, 2)
Converting haps to FASTA sequence | upstream
Removing 85 alleles with a missing call
Multiple alignment FASTA written 

loading haplotypes | downstream | 2L:30535767-30555767
downstream 2L:30535767-30555767 (5131, 2431, 2)
Converting haps to FASTA sequence | downstream
Removing 64 alleles with a missing call
Multiple alignment FASTA written 

loading haplotypes | collinear | 2L:44990000-45010000
collinear 2L:44990000-45010000 (5061, 2431, 2)
Converting haps to FASTA sequence | collinear
Removing 84 alleles with a missing call
Multiple alignment FASTA written 



## Running IQTree
You may prefer to do this outside of the jupyter notebook, it can take a while :) 

In [ ]:
#for name in names:
#   !iqtree -s {locus_name}_{name}.fasta -B 10000

### Testing the speed of numba  vs naive python implementation

A function which generated a FASTA sequence from the haplotype alleles was a bottleneck in computational time, so I made a numba implementation, and this turned out to be suuuuuper quick. 

In [42]:
def haps_to_fasta_naive(haplos, var_alleles):
    seq_list = [] 
    for i in range(haplos.shape[1]):
        seq_arr = []

        for gn, allele in enumerate(haplos[:,i]):
            seq_arr.append(var_alleles[gn][allele])
        
        seq_list.append(np.array(seq_arr))
    return(np.vstack(seq_list))


def haps_to_fasta_eric(haplos, var_alleles):
    return(var_alleles[range(var_alleles.shape[0]), np.transpose(haplos)])

@njit
def haps_to_fasta_eric_numba(haplos, var_alleles):
    return(var_alleles[np.arange(var_alleles.shape[0]), np.transpose(haplos)])

In [1]:
#
#haps_to_fasta_eric_numba(haplos.values, var_alleles.values)

In [17]:
#haps = haps['focal']

In [18]:
# transform xarray into haplotype array
#haplos = allel.GenotypeArray(haps['call_genotype'].compute()).to_haplotypes()
# extract ref and alt alleles array
#var_alleles = haps['variant_allele'].compute()

In [26]:
#%%timeit
#seq_arr = haps_to_fasta_naive(haplos.values, var_alleles.values)

7.69 s ± 251 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [25]:
#%%timeit
#seq_arr = haps_to_fasta_numba(haplos.values, var_alleles.values)
#seq_arr = np.vstack(seq_arr)

75.6 ms ± 1.35 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [24]:
#%%timeit
#haps_to_fasta_eric(haplos.values, var_alleles.values)

90.1 ms ± 1.08 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
